In [ ]:
import numpy as np
import nibabel as nib
import random

def generate_3d_volume(image_size, radius, center_sphere):
    # Create a 3D grid
    x = np.linspace(0, 1, image_size[0])
    y = np.linspace(0, 1, image_size[1])
    z = np.linspace(0, 1, image_size[2])
    x, y, z = np.meshgrid(x, y, z, indexing='ij')  # Ensure correct indexing for the grid

    # Sphere mask using the fixed radius and center position
    sphere_mask = ((x - center_sphere[0])**2 + (y - center_sphere[1])**2 + (z - center_sphere[2])**2) < (radius / image_size[0])**2


    # Initialize volume with grey for background
    volume = np.full(image_size, 255)  # Set entire volume to grey

    # Set the sphere to white
    volume[sphere_mask] = 128  # Make the sphere white

    return volume

# Parameters
image_size = (16, 16, 16)  # Example size of the 3D volume, adjusted for practicality
radius = 4  # Example radius, scaled to the example volume size
num_samples = 1000  # Number of samples for demonstration

volumes = []
for i in range(num_samples):
    # Generate a random center for the sphere within the volume
    sphere_center = (
        random.uniform(radius / image_size[0], 1 - radius / image_size[0]),
        random.uniform(radius / image_size[1], 1 - radius / image_size[1]),
        random.uniform(radius / image_size[2], 1 - radius / image_size[2])
        )

    # Generate 3D volume
    volume = generate_3d_volume(image_size, radius, sphere_center)
    volumes.append(volume)
    
# Save all volumes in a single numpy file
# Saving the generated volumes to a .npz file
np.savez('synthetic_dataset.npz', *volumes)

# Example of saving one volume to a NIfTI file (for demonstration)
nifti_image = nib.Nifti1Image(volumes[0].astype(np.float32), np.eye(4))
nifti_file_path = 'input.nii.gz'
nib.save(nifti_image, nifti_file_path)


In [ ]:
import os
import tensorflow as tf
import shutil
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Conv3DTranspose, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, Callback
import h5py
from datetime import datetime
import numpy as np
import pdb
import matplotlib.pyplot as plt
import random
from tqdm.keras import TqdmCallback



# Load your npz file
loaded_data = np.load('synthetic_dataset.npz')
num_samples = len(loaded_data.files)
sample_fraction = 1
num_samples_to_use = int(num_samples * sample_fraction)


# Use the loaded_data dictionary to get the volumes
train_x = np.array([loaded_data[f'arr_{i}'] for i in range(num_samples_to_use)])
print(list(set([np.max(train_x[i]) for i in range(len(train_x))])))
print(train_x.shape)
# exit(1)

# Reshape train_x to add a channel dimension, assuming grayscale images so only 1 channel is added
train_x = np.expand_dims(train_x, axis=-1)
print("New train_x shape with channel dimension:", train_x.shape)

# Define autoencoder architecture
def define_autoencoder(input_shape):

    input_layer = Input(shape=input_shape)
    # print('input_shape in auto encoder',input_shape)
    
    # Encoder
    encoder = Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same')(input_layer)
    print('####encoder###',encoder)
    # exit(1)
    
    encoder = BatchNormalization()(encoder)
    encoder = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(encoder)
    
    print('####encoder###',encoder)
    
    
    encoder = Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding='same')(encoder)
    print('####encoder###',encoder)
    
    encoder = BatchNormalization()(encoder)
    encoder = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(encoder)
    # print('####encoder###',encoder)
    print('####output_layer###',encoder.shape)
    
    
    
    # Decoder
    decoder = Conv3DTranspose(128, kernel_size=(3, 3, 3), activation='relu', padding='same')(encoder)
    decoder = BatchNormalization()(decoder)
    decoder = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(decoder)
    print('####output_layer###',decoder.shape)
    
    
    
    decoder = Conv3DTranspose(64, kernel_size=(3, 3, 3), activation='relu', padding='same')(decoder)
    decoder = BatchNormalization()(decoder)
    decoder = tf.keras.layers.UpSampling3D(size=(2, 2, 2))(decoder)
    
    print('####output_layer###',decoder.shape)
    
    
    # Output layer with the same number of channels as input data
    output_layer = Conv3D(1, kernel_size=(3, 3, 3), activation='sigmoid', padding='same')(decoder)
    output_layer = output_layer * 255
    print('####output_layer###',output_layer.shape)
    
    
    
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    autoencoder.compile(optimizer= optimizer, loss= tf.keras.losses.MeanSquaredError())  # Adjust loss function if needed
    
    # print('####autoencoder###',autoencoder)
    
    return autoencoder




# Instantiate and compile autoencoder
input_shape = train_x.shape[1:]
print("the last input shape", train_x.shape[1:])

autoencoder = define_autoencoder(input_shape)

# Create a tf.data.Dataset from the numpy array
dataset = tf.data.Dataset.from_tensor_slices(train_x)  # Changed line
batch_size = 10  # Added line
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)  # Changed line





# Train the autoencoder using the dataset
autoencoder.fit(dataset.map(lambda x: (x, x)), epochs=100)  # Changed line

autoencoder.save('autoencoder_model.h5')







In [ ]:

autoencoder = tf.keras.models.load_model('autoencoder_model.h5')

    
print(train_x[2].shape)
# validation_data

decoded_imgs = autoencoder.predict(train_x)
# Assuming custom_dataset and autoencoder are defined
for i in range(1):
    # Visualize first three volumes
    input_volume = train_x[i]
    
    reconstructed_volume = decoded_imgs[i]
        # Convert the numpy array to a NIfTI image
    nifti_image = nib.Nifti1Image(reconstructed_volume, np.eye(4))

    # Save the NIfTI image to a file
    nifti_file_path = 'rec.nii.gz'
    nib.save(nifti_image, nifti_file_path)
   